In [667]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn import linear_model
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import normalize
from sklearn.model_selection import cross_val_predict
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from sklearn.svm import SVR
#from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_log_error
from sklearn.preprocessing import PolynomialFeatures
from scipy.stats import boxcox
import pandas_profiling
import cluster_fun
from sklearn.model_selection import cross_validate

In [1013]:
rawdata = pd.read_csv('cleandata_v2.csv')
rawdata.head()

,Unnamed: 0,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,0,1,60,RL,65.0,8450,Pave,No,3,Lvl,...,0,0,0,No,0,2,2008,WD,Normal,208500.0
1,1,2,20,RL,80.0,9600,Pave,No,3,Lvl,...,0,0,0,No,0,5,2007,WD,Normal,181500.0
2,2,3,60,RL,68.0,11250,Pave,No,2,Lvl,...,0,0,0,No,0,9,2008,WD,Normal,223500.0
3,3,4,70,RL,60.0,9550,Pave,No,2,Lvl,...,0,0,0,No,0,2,2006,WD,Abnorml,140000.0
4,4,5,60,RL,84.0,14260,Pave,No,2,Lvl,...,0,0,0,No,0,12,2008,WD,Normal,250000.0


In [1014]:
coltodrop = ['Unnamed: 0', "LotFrontage"]
data = rawdata.drop(coltodrop, axis= 1)

In [1015]:
categorical_columns_all = ['MSSubClass','MSZoning','MasVnrType','PoolQC','MiscFeature','Street','Alley','LotShape',\
                           'LandContour','Utilities','LotConfig','LandSlope','Neighborhood','Condition1','Condition2',\
                           'BldgType','HouseStyle','RoofStyle','RoofMatl','Exterior1st','Exterior2nd','MasVnrType',\
                           'ExterQual','ExterCond','Foundation','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1',\
                           'BsmtFinType2','Heating','HeatingQC','CentralAir','Electrical','KitchenQual','Functional',\
                           'FireplaceQu','GarageType','GarageFinish','GarageQual','GarageCond','PavedDrive','Fence',\
                           'SaleType','SaleCondition','MoSold','YrSold']

numerical_columns = ['LotArea','OverallQual','MasVnrArea','PoolArea','OverallCond','YearBuilt','YearRemodAdd',\
                         'MasVnrArea','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','1stFlrSF','2ndFlrSF',\
                         'LowQualFinSF','GrLivArea','BsmtFullBath','BsmtHalfBath','FullBath','HalfBath','BedroomAbvGr',\
                         'KitchenAbvGr','TotRmsAbvGrd','Fireplaces','GarageYrBlt','GarageCars','GarageArea','WoodDeckSF',\
                         'OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch','PoolArea','MiscVal']


In [1016]:
colToCluster_all = rawdata[categorical_columns_all].agg("nunique").sort_values(ascending = False).head(10)
colToCluster_all.head()
colToCluster_all = pd.DataFrame(colToCluster_all).reset_index()
colToCluster_all.columns = ['Col', 'n']
colToCluster_all

,Col,n
0,Neighborhood,25
1,Exterior2nd,16
2,MSSubClass,16
3,Exterior1st,15
4,MoSold,12
5,Condition1,9
6,SaleType,9
7,Condition2,8
8,RoofMatl,8
9,HouseStyle,8


RUN K MEANS ON MULTIPLE COLUMNS AND REPLACE CATEGORICAL VARIABLES

In [1017]:
trainData = data[:1460]

In [1018]:
cols = ['Neighborhood', 'Exterior2nd', "Exterior1st", 'MoSold']
categorical_columns_all = pd.DataFrame(categorical_columns_all)
fltr = categorical_columns_all.isin(cols)
categorical_columns_all = np.array(categorical_columns_all)
newCatCol = categorical_columns_all[-fltr]
newCatCol

array(['MSSubClass', 'MSZoning', 'MasVnrType', 'PoolQC', 'MiscFeature',
       'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'RoofStyle', 'RoofMatl', 'MasVnrType', 'ExterQual',
       'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure',
       'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC',
       'CentralAir', 'Electrical', 'KitchenQual', 'Functional',
       'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'Fence', 'SaleType', 'SaleCondition',
       'YrSold'], dtype=object)

In [1019]:
colToCluster = colToCluster_all.loc[colToCluster_all.Col.isin(cols)]
colToCluster.n = round(colToCluster.n/2).astype("int")
colToCluster = colToCluster.reset_index()

In [1020]:
for i in range(len(colToCluster)):
    n = colToCluster.loc[i,"n"]
    col = colToCluster.loc[i,"Col"]
    kdf = cluster_fun.cluster(5, trainData[[col, 'SalePrice']]) 
    data = cluster_fun.kReplace(data, kdf, col, 'SalePrice')

In [978]:
data.MSSubClass.value_counts()

#MSSUBCLASS GIVES NoneType' object is not subscriptable ERROR WHYYYYY

20     1079
60      575
50      287
120     182
30      139
160     128
70      128
80      118
90      109
190      61
85       48
75       23
45       18
180      17
40        6
150       1
Name: MSSubClass, dtype: int64

GET DUMMIES AND NORMALIZE NUMERICAL COLUMNS

In [763]:

data[numerical_columns] = data[numerical_columns].apply(lambda x: (x - x.min()) / (x.max() - x.min()))
data = pd.get_dummies(data, prefix_sep = '_', columns=newCatCol, drop_first = True)


SPLIT TRAIN AND TEST

In [764]:
xTr = data.loc[:1459,].drop("SalePrice", axis = 1)
yTr = data.loc[:1459,].SalePrice

yTrLog = np.log(yTr)

yTrPsq = yTr/rawdata['GrLivArea'][:1460]
xTrPsq = xTr.drop('GrLivArea', axis = 1)

yTrLogPsq = np.log(yTrPsq)
xTrLogPsq = xTrPsq

xTe = data.loc[1460:,].drop("SalePrice", axis = 1)
xTePsq = xTe.drop('GrLivArea', axis = 1)


MARIO'S EVALUATE MODEL FUNCTION

In [765]:
def evaluate_model(model, x, y, folds):
    print("Accuracy: ",round(model.score(x, y),4))
    scores = -cross_val_score(model, x, y, scoring='neg_mean_squared_error', cv = folds)
    ypred = abs(cross_val_predict(model, x, y, cv = folds, n_jobs = -1))
    print("RMSLE = ", round(np.sqrt(mean_squared_error(ypred, y )),4) )
    print("RMSLE = ", round(np.sqrt(mean_squared_error(np.log(ypred), np.log(y) )),4) )
   # print("Cross-Val Score =",round(np.mean(scores**.5)))
    

OLS

In [766]:
print('y')
ols = linear_model.LinearRegression()
x = xTr
y = yTr
ols.fit(x, y)

evaluate_model(ols, x, y, 5)

print('\n Log y ')
ols = linear_model.LinearRegression()
x = xTr
y = yTrLog
ols.fit(x, y)

evaluate_model(ols, x, y, 5)


print('\n y/GLA')

ols = linear_model.LinearRegression()
x = xTrPsq
y = yTrPsq
ols.fit(x, y)

evaluate_model(ols, x, y, 5)



print('\n Log(y/GLA)')

ols = linear_model.LinearRegression()
x = xTrLogPsq
y = yTrLogPsq

ols.fit(x, y)


evaluate_model(ols, x, y, 5)


'''
y
Accuracy:  0.9345
RMSLE =  37669.2154
RMSLE =  0.2444

 Log y 
Accuracy:  0.9459
RMSLE =  0.1906
RMSLE =  0.0164

 y/GLA
Accuracy:  0.872
RMSLE =  16.9406
RMSLE =  0.1778

 Log(y/GLA)
Accuracy:  0.8824
RMSLE =  0.1637
RMSLE =  0.0368
'''

y
Accuracy:  0.9218
RMSLE =  39420.6284
RMSLE =  0.2493

 Log y 
Accuracy:  0.9378
RMSLE =  0.1985
RMSLE =  0.0172

 y/GLA
Accuracy:  0.8506
RMSLE =  16.9693
RMSLE =  0.1637

 Log(y/GLA)
Accuracy:  0.8626
RMSLE =  0.1727
RMSLE =  0.0397


'\ny\nAccuracy:  0.9345\nRMSLE =  37669.2154\nRMSLE =  0.2444\n\n Log y \nAccuracy:  0.9459\nRMSLE =  0.1906\nRMSLE =  0.0164\n\n y/GLA\nAccuracy:  0.872\nRMSLE =  16.9406\nRMSLE =  0.1778\n\n Log(y/GLA)\nAccuracy:  0.8824\nRMSLE =  0.1637\nRMSLE =  0.0368\n'

RIDGE

In [767]:
print('Log y')
ridge = linear_model.RidgeCV(alphas=np.arange(0.05,4,0.01),normalize=True)
ridge.fit(xTr, yTrLog)
yp = ridge.predict(xTr)

print(np.sqrt(np.mean((yp - np.log(yTr))**2)))


print('\n y/GLA')
ridge = linear_model.RidgeCV(alphas=np.arange(0.05,4,0.01),normalize=True)
ridge.fit(xTrPsq, yTrPsq)
yp = ridge.predict(xTrPsq)


print(np.sqrt(np.mean((np.log(yp) - np.log(yTrPsq))**2)))



print('\n log(y/GLA)')
ridge = linear_model.RidgeCV(alphas=np.arange(0.05,4,0.01),normalize=True)
ridge.fit(xTrLogPsq, yTrLogPsq)
yp = ridge.predict(xTrPsq)

print(np.sqrt(np.mean((yp - yTrLogPsq)**2)))


yp = ridge.predict(xTePsq)
y1 = np.exp(yp)* rawdata['GrLivArea'][1460:]
pd.DataFrame(y1).to_csv("ridge2.csv")


#print(rawdata['GrLivArea'][1460:])
#ridge = linear_model.Ridge(ridge.alpha_,normalize=True)
#ridge.fit(xTrPsq, yTrPsq)
#evaluate_model(ridge,xTrPsq, yTrPsq, 5)


#Best alpha =  0.5500000000000002
#Accuracy:  0.8851
#RMSLE =  0.1492
#Cross-Val Score = 33022.0 with neighborhoods grouped


Log y
0.127763086736745

 y/GLA
0.11857914898738613

 log(y/GLA)
0.11040040989482557


LASSO

In [677]:
x = xTrLogPsq[:]
y = yTrLogPsq[:]

lasso = linear_model.LassoCV(eps=0.001, n_alphas=100, cv=5, normalize=True, max_iter=10000)

lasso.fit(x, y)
print("Best alpha = ",lasso.alpha_)
alpha=lasso.alpha_
lasso = linear_model.Lasso(alpha=lasso.alpha_, normalize=True)
lasso.fit(x, y)
evaluate_model(lasso, x, y, 5)
yp = lasso.predict(x)
print(np.sqrt(np.mean((yp - yTrLogPsq)**2)))


scores = cross_validate(lasso, x, y, cv=5,scoring=('r2'),
                        return_train_score=True)

print(scores['test_score']) 
#Best alpha =  11.174320253763256
#Accuracy:  0.9147
#RMSLE =  0.154
#Cross-Val Score = 32724.0 with neighborhoods grouped

Best alpha =  0.00010093472095098882
Accuracy:  0.8124
RMSLE =  0.1401
RMSLE =  0.031
0.11954266821111784
[0.77840918 0.71414186 0.71026317 0.79504175 0.71788326]


In [678]:
xTrLogPsq.head()

,Id,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,MoSold_7,MoSold_8,MoSold_9,MoSold_10,MoSold_11,MoSold_12,YrSold_2007,YrSold_2008,YrSold_2009,YrSold_2010
0,1,0.033420,0.666667,0.500,0.949275,0.883333,0.12250,0.125089,0.0,0.064212,...,0,0,0,0,0,0,0,1,0,0
1,2,0.038795,0.555556,0.875,0.753623,0.433333,0.00000,0.173281,0.0,0.121575,...,0,0,0,0,0,0,1,0,0,0
2,3,0.046507,0.666667,0.500,0.934783,0.866667,0.10125,0.086109,0.0,0.185788,...,0,0,1,0,0,0,0,1,0,0
3,4,0.038561,0.666667,0.500,0.311594,0.333333,0.00000,0.038271,0.0,0.231164,...,0,0,0,0,0,0,0,0,0,0
4,5,0.060576,0.777778,0.500,0.927536,0.833333,0.21875,0.116052,0.0,0.209760,...,0,0,0,0,0,1,0,1,0,0
